# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import shuffle_arrays_unison
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter('ignore')

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


## Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
#загружаем данные и обрабоатываем их для алгоритма
iris = load_iris()
iris= pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                 columns= iris['feature_names'] + ['target'])
X_train, X_test, y_train, y_test = train_test_split(iris.iloc[:,0:4], pd.DataFrame(iris.target), test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = X_train.reset_index(drop = True), X_test.reset_index(drop = True), y_train.reset_index(drop = True), y_test.reset_index(drop = True)
X_train['target'] = y_train
X_test['target'] = y_test

### Decision tree

In [ ]:
class Node(object):
	def __init__(self, attribute, threshold):
		self.attr = attribute
		self.thres = threshold
		self.left = None
		self.right = None
		self.leaf = False
		self.predict = None

#поиск параметра разделения
def select_threshold(df, attribute):
  grid = np.linspace(df[attribute].min(), df[attribute].max(), 10)
  max_ig = float("-inf")
  thres_val = 0
	#перебрать все пороговые значения, находящиеся между минимальным и максимальным значением атрибута
  for t in grid:
    ig = info_gain(df, attribute, t)
    if ig > max_ig:
      max_ig = ig
      thres_val = t	
  return thres_val


#расширение для формулы информационного прироста
def remainder(df, df_subsets):
	remainder = float(0)
	for df_sub in df_subsets:
		if df_sub.shape[0] > 1:
			remainder += float(df_sub.shape[0]/df.shape[0])*entropy(df_sub.target)
	return remainder

#расчет IG для конкретного признака
def info_gain(df, attribute, threshold):
  sub_1 = df[df[attribute] < threshold]
  sub_2 = df[df[attribute] >= threshold]
  ig = entropy(df.target) - remainder(df, [sub_1, sub_2])
  return ig


#выбираем лучший атрибут и соответствующий параметр разделения
def choose_attr(df):
  best_IG = float("-inf")
  best_attr = None
  best_t = 0
  for attr in df.drop(['target'], axis = 1).columns:
    t = select_threshold(df, attr)
    ig = info_gain(df, attr, t)
    if ig > best_IG:
      best_IG = ig
      best_attr = attr
      best_t = t
  return best_attr, best_t

#расчет энтропии
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

#само дерево
def build_tree(df, depth = 0, max_depth=5, min_samples_split=5):
  purity = len(pd.unique(df.target))
  X = df.drop(['target'], axis = 1)
  y = df.target
  if depth == max_depth or purity == 1 or df.shape[0] < min_samples_split:
    leaf = Node(None, None)
    leaf.leaf = True
    leaf.predict= np.argmax(np.bincount(y))
    return leaf
  else: 
    depth += 1
    best_attr, threshold = choose_attr(df)
    tree = Node(best_attr, threshold)
    sub_1 = df[df[best_attr] < threshold]
    sub_2 = df[df[best_attr] >= threshold]
    tree.left = build_tree(sub_1,depth)
    tree.right = build_tree(sub_2, depth)
    return tree

def predict(node, row_df):
	if node.leaf:
		return node.predict
	if row_df[node.attr] < node.thres:
		return predict(node.left, row_df)
	elif row_df[node.attr] >= node.thres:
		return predict(node.right, row_df)

def predictions(root, df):
  prediction = [predict(root, row)  for index, row in df.iterrows()]
  return prediction


root = build_tree(X_train)

print("Accuracy of test data")
print(accuracy_score(y_test, predictions(root, X_test)))

Accuracy of test data
0.98


## Задание 4.2

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

###  Random forest

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


In [ ]:
#Random Forest
def bootstrap(X, y, n_estimators, max_features, max_depth, min_samples_leaf):
  forest = []
  n_sub_samples = np.random.randint(1, len(X))
  for i in range(n_estimators):
    indexes = np.random.randint(0, len(X), n_sub_samples).tolist()
    X_subset = X.loc[indexes]
    y_subset = y.loc[indexes]
    tree = DecisionTreeClassifier(criterion = "entropy", max_features = max_features, max_depth = max_depth, min_samples_leaf = min_samples_leaf)
    tree = tree.fit(X_subset, y_subset)
    forest.append(tree)
  return forest

def predict_y(model, X):
  n_samples = X.shape[0]
  predictions = np.empty([len(model), n_samples])
  features_weights = []
  for i in range(len(model)):
    predictions[i] = model[i].predict(X)
    features_weights.append(model[i].feature_importances_)
  #отбор признаков
  features_name = model[0].feature_names_in_
  features = pd.DataFrame(np.row_stack(features_weights), columns = [features_name])
  best_features = []
  for name in features.columns:
     best_features.append(features[name].mean())

  feature_importances = pd.DataFrame(best_features, index = [name[0] for name in features.columns])

  return mode(predictions)[0][0], feature_importances

#проверка на iris
model= bootstrap(X_train.drop(['target'], axis = 1), X_train.target, n_estimators = 32, max_features="sqrt", max_depth = 6, min_samples_leaf = 1)
y_pred,_ = predict_y(model, X_test.drop(['target'], axis = 1))
print("Accuracy of test data")
print(accuracy_score(y_test, y_pred))

Accuracy of test data
1.0


### Анализ данных

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

In [ ]:
#обработка данных
churn = pd.read_csv("https://media.githubusercontent.com/media/daryabaldina1702/Mathematical_methods_of_machine_learning/main/datasets/churn.csv", sep = ",")
churn = churn.drop(columns=['RowNumber','CustomerId', 'Surname']) #удаляем безинформативные столбцы
churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#энкодим категориальные признаки
churn_encode = churn.copy()
churn_encode["Gender"] = LabelEncoder().fit_transform(churn_encode["Gender"]) 
churn_encode = pd.get_dummies(churn_encode, columns=["Geography"], prefix=["geo"]) #OneHotEncoder для избежания неверной интерпретации алгоритма
churn_encode.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,geo_France,geo_Germany,geo_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [ ]:
#разделение выборки 
X = churn_encode.drop(columns = "Exited")
y = pd.DataFrame(churn_encode.Exited)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=45)
X_train, X_test, y_train, y_test = X_train.reset_index(drop = True), X_test.reset_index(drop = True), y_train.reset_index(drop = True), y_test.reset_index(drop = True)

In [ ]:
#обучение Random Forest
model= bootstrap(X_train, y_train, n_estimators = 110, max_features="sqrt", max_depth = 300, min_samples_leaf = 1)
y_pred,features  = predict_y(model, X_test)
print("Accuracy of test data")
print(accuracy_score(y_test, y_pred))

Accuracy of test data
0.8521212121212122


In [ ]:
#важные признаки (топ-5)
features = features.rename(columns = {0:"importance"})
features = features.sort_values(by = "importance", ascending = False)
features.head()

,importance
Age,0.236545
EstimatedSalary,0.148770
CreditScore,0.147526
Balance,0.140285
NumOfProducts,0.124381


#### Вывод

По результам обучения видно, что наиболее важные признаки в используемом наборе данных:

*   возраст(Age), что очевидно, ведь пожилые люди с меньшей вероятностью захотятсменить банк, чем молодые;
*   кредитный рейтинг(CreditScore), которвй может повлиять на отток клиентов, поскольку клиент с более высоким кредитным рейтингом с меньшей вероятностью покинет банк;
*   расчетная зарплата(EstimatedSalary), что также логично, ведь люди с более низкой зарплатой с большей вероятностью уйдут из банка;
*   остаток на счете (Balance)(также, как и EstimatedSalary); 
*   количество продуктов, которые клиент приобрел через банк(NumOfProducts) — чем их больше, тем меньше вероятность, что клиент захочет уйти из банка
 